In [1]:
# # Mount GDRIVE jika menggunakan Google Collab
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# PATH = '/content/drive/MyDrive/Tugas Akhir - Iki/TA2/Source Code/program'
# PATH_1 = '/backend_api/data/topic classification data/'
PATH = ''
PATH_1 = './backend_api/data/topic classification data/'

In [3]:
# !pip install transformers

In [4]:
import pandas as pd

data = pd.read_csv(PATH + PATH_1 + 'topic_classification_balanced.csv', delimiter=';')
data

,kategori,isi_preprocessed
0,hot,cerai hidup kacau cerita hijrah peggy melati s...
1,ekonomi,orang kaya orang kaya beneran
2,kesehatan,sebar positif virus corona indonesia
3,hot,cerita mawar jongh hadap serang netizen
4,hot,grande blackpink muncul lady gaga
...,...,...
28652,teknologi,browser vivaldi sedia ios bawa fitur fokus pri...
28653,teknologi,meta headset generasi pasang kacamata pintar s...
28654,teknologi,samsung geser jendela seri galaxy februari jan...
28655,teknologi,dyson floorcare tech letak inovasi desain rama...


In [5]:
data["kategori"].value_counts()

kategori
hot                   6000
ekonomi               6000
berita dan politik    6000
kesehatan             4983
teknologi             2993
olahraga              2681
Name: count, dtype: int64

In [6]:
data = data.dropna().reset_index(drop=True)

In [7]:
data.isnull().sum()

kategori            0
isi_preprocessed    0
dtype: int64

In [8]:
data.columns = ['label', 'text']
data

,label,text
0,hot,cerai hidup kacau cerita hijrah peggy melati s...
1,ekonomi,orang kaya orang kaya beneran
2,kesehatan,sebar positif virus corona indonesia
3,hot,cerita mawar jongh hadap serang netizen
4,hot,grande blackpink muncul lady gaga
...,...,...
28643,teknologi,browser vivaldi sedia ios bawa fitur fokus pri...
28644,teknologi,meta headset generasi pasang kacamata pintar s...
28645,teknologi,samsung geser jendela seri galaxy februari jan...
28646,teknologi,dyson floorcare tech letak inovasi desain rama...


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data

,label,text
0,2,cerai hidup kacau cerita hijrah peggy melati s...
1,1,orang kaya orang kaya beneran
2,3,sebar positif virus corona indonesia
3,2,cerita mawar jongh hadap serang netizen
4,2,grande blackpink muncul lady gaga
...,...,...
28643,5,browser vivaldi sedia ios bawa fitur fokus pri...
28644,5,meta headset generasi pasang kacamata pintar s...
28645,5,samsung geser jendela seri galaxy februari jan...
28646,5,dyson floorcare tech letak inovasi desain rama...


In [10]:
le.inverse_transform([0,1,2,3,4,5])

array(['berita dan politik', 'ekonomi', 'hot', 'kesehatan', 'olahraga',
       'teknologi'], dtype=object)

In [11]:
data['text'] = data['text'].astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28648 entries, 0 to 28647
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   28648 non-null  int32 
 1   text    28648 non-null  object
dtypes: int32(1), object(1)
memory usage: 335.8+ KB


# Training Model

In [12]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
N_CLASSES = 3

## Make dataset

In [13]:
from datasets import Dataset

d:\Kuliah\Rifky - Full program setelah revisi\virtualenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 28648
})

In [15]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset['train']
val_test_dataset  = dataset['test']
val_test_dataset = val_test_dataset.train_test_split(test_size=0.2, seed=42)
val_dataset, test_dataset = val_test_dataset['train'], val_test_dataset['test']
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['label', 'text'],
     num_rows: 22918
 }),
 Dataset({
     features: ['label', 'text'],
     num_rows: 4584
 }),
 Dataset({
     features: ['label', 'text'],
     num_rows: 1146
 }))

## Training

In [16]:
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# model_name = "cahya/distilbert-base-indonesian"
# tokenizer = DistilBertTokenizer.from_pretrained(model_name)
# model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=6)  # num_labels sesuai jumlah kelas

In [17]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_name = "cahya/distilbert-base-indonesian"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=6)  # num_labels sesuai jumlah kelas

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def tokenize(batch):
    return tokenizer(batch['text'], max_length=MAX_LEN, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 4584/4584 [00:00<00:00, 5117.16 examples/s]


In [19]:
class_weights = (1 - (data["label"].value_counts().sort_index() / len(data))).values
class_weights

array([0.79063111, 0.79063111, 0.79063111, 0.82613097, 0.90645071,
       0.89552499])

In [20]:
import torch

class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.7906, 0.7906, 0.7906, 0.8261, 0.9065, 0.8955], device='cuda:0')

In [21]:
try:
  train_dataset = train_dataset.rename_column("label", "labels")
  val_dataset = val_dataset.rename_column("label", "labels")
except Exception as e:
  print(e)

val_dataset, train_dataset

(Dataset({
     features: ['labels', 'text', 'input_ids', 'attention_mask'],
     num_rows: 4584
 }),
 Dataset({
     features: ['labels', 'text', 'input_ids', 'attention_mask'],
     num_rows: 22918
 }))

In [22]:
from transformers import Trainer, TrainingArguments
from torch import nn

In [23]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def compute_metrics(pred):
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # acc = accuracy_score(labels, preds)
    # f1 = f1_score(labels, preds, average='weighted')
    # return {
    #     'accuracy': acc,
    #     'f1': f1,
    # }
    labels = pred.label_ids
    if isinstance(pred.predictions, tuple):
        preds_logits = pred.predictions[0]
    else:
        preds_logits = pred.predictions

    # Apply argmax to the logits to get the predicted class indices
    preds = preds_logits.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1': f1,
    }

In [24]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir=PATH+"./my_indodistilbert2_topic_classification",
    eval_strategy="epoch",  # must evaluate to compute metrics
    logging_strategy="epoch",     # must log during evaluation
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    learning_rate=2e-5,
)

# training_args = TrainingArguments(
#     output_dir=PATH+"./my_indodistilbert2_topic_classification",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=2,
#     gradient_checkpointing=True,
#     num_train_epochs=4,
#     fp16=True,
#     load_best_model_at_end=True,
#     save_total_limit=2,
#     dataloader_pin_memory=True,
# )


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# trainer = WeightedLossTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\Kocheng\AppData\Local\Temp\ipykernel_112692\1412765637.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


In [29]:
# metrics = trainer.evaluate()
# print(metrics)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.344000,0.610786
2,0.281800,0.591135
3,0.185400,0.746998
4,0.100400,0.802885


TrainOutput(global_step=11460, training_loss=0.2248433667327721, metrics={'train_runtime': 1178.3268, 'train_samples_per_second': 77.798, 'train_steps_per_second': 9.726, 'total_flos': 575043703634736.0, 'train_loss': 0.2248433667327721, 'epoch': 4.0})

In [31]:
model.save_pretrained(PATH+"./my_indodistilbert2_topic_classification")
tokenizer.save_pretrained(PATH+"./my_indodistilbert2_topic_classification")

('./my_indodistilbert2_topic_classification\\tokenizer_config.json',
 './my_indodistilbert2_topic_classification\\special_tokens_map.json',
 './my_indodistilbert2_topic_classification\\vocab.txt',
 './my_indodistilbert2_topic_classification\\added_tokens.json')

In [32]:
import pandas as pd
import torch

# tokenized_input = tokenizer(test_dataset['text'], return_tensors="pt", truncation=True, padding=True)
# tokenized_input
test_dataset_tokenized = test_dataset.map(tokenize, batched=True)
test_dataset_tokenized

Map: 100%|██████████| 1146/1146 [00:00<00:00, 5997.71 examples/s]


Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1146
})

In [33]:
prediction = trainer.predict(test_dataset_tokenized)
prediction

PredictionOutput(predictions=(array([[-0.22692871, -3.5820312 ,  6.0273438 , -0.7558594 , -2.8183594 ,
        -2.40625   ],
       [-1.859375  , -2.7871094 , -1.9589844 , -2.3789062 ,  6.0234375 ,
        -1.5019531 ],
       [-1.9091797 , -1.9785156 , -3.0078125 , -3.2929688 , -1.9902344 ,
         7.8164062 ],
       ...,
       [-0.8720703 , -2.5976562 ,  6.7734375 , -0.95214844, -2.9472656 ,
        -3.6445312 ],
       [ 0.24206543, -2.2324219 , -2.6503906 ,  6.4453125 , -3.3125    ,
        -3.546875  ],
       [-0.62353516, -3.0058594 ,  6.4765625 , -1.0576172 , -2.2128906 ,
        -3.5625    ]], dtype=float32), (array([[[-4.0591398e-01,  1.7262024e-01,  5.5808771e-01, ...,
          7.9672700e-01, -1.4772902e+00,  6.6047162e-02],
        [ 7.9941429e-02,  9.0035307e-01, -1.0814452e+00, ...,
          1.4536324e+00, -1.3290514e-01, -1.4538318e-01],
        [-5.2907962e-01, -2.9016972e-01,  4.6954501e-01, ...,
          1.2344952e+00,  3.0325418e-02,  3.5162829e-02],
        ..

In [34]:
compute_metrics(prediction)

{'accuracy': 0.8481675392670157, 'f1': 0.8471582733101456}

In [124]:
# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    x = x.transpose()
    e_x = np.exp(x - np.max(x))
    r = e_x / e_x.sum(axis=0) # only difference
    return r.transpose()

In [129]:
arr = np.array([[1, 2, 3, 6],
                     [2, 4, 5, 6],
                     [3, 8, 7, 6]])
print(arr.shape)
print(softmax(arr))
print(np.sum(softmax(arr), axis=-1))

(3, 4)
[[0.00626879 0.01704033 0.04632042 0.93037047]
 [0.01203764 0.08894682 0.24178252 0.65723302]
 [0.00446236 0.66227241 0.24363641 0.08962882]]
[1. 1. 1.]


In [140]:
preds_logits = prediction.predictions

print(preds_logits.argmax(-1))
print(preds_logits.max(-1))
print(max(preds_logits.max(-1)))
print(min(preds_logits.max(-1)))
print("")

print(softmax(preds_logits))
print(torch.softmax(torch.from_numpy(preds_logits), dim=-1))
print(np.sum(softmax(preds_logits), axis=-1))
print(softmax(preds_logits).max(-1))
print(softmax(preds_logits).min(-1))
print(max(softmax(preds_logits).max(-1)))
print(min(softmax(preds_logits).max(-1)))

[2 4 5 ... 2 3 2]
[4.7148438 6.125     5.3710938 ... 5.0351562 5.3398438 5.015625 ]
6.1640625
0.4819336

[[1.0004894e-02 5.4360996e-04 9.7712767e-01 9.6639888e-03 1.8734497e-03
  7.8632700e-04]
 [5.1354221e-04 3.4849948e-04 3.6665375e-04 3.5746139e-04 9.9808902e-01
  3.2483725e-04]
 [1.0417288e-03 4.6588990e-04 4.3372596e-03 1.0447850e-03 5.8094290e-04
  9.9252939e-01]
 ...
 [2.2226050e-03 1.1954979e-03 9.8961180e-01 2.7432111e-03 3.6698284e-03
  5.5704504e-04]
 [5.1679085e-03 9.2961383e-04 1.3846557e-03 9.9194753e-01 3.2252335e-04
  2.4777066e-04]
 [5.3393878e-03 5.1573134e-04 9.8680389e-01 4.9170740e-03 1.9031205e-03
  5.2079244e-04]]
tensor([[1.0005e-02, 5.4361e-04, 9.7713e-01, 9.6640e-03, 1.8734e-03, 7.8633e-04],
        [5.1354e-04, 3.4850e-04, 3.6665e-04, 3.5746e-04, 9.9809e-01, 3.2484e-04],
        [1.0417e-03, 4.6589e-04, 4.3373e-03, 1.0448e-03, 5.8094e-04, 9.9253e-01],
        ...,
        [2.2226e-03, 1.1955e-03, 9.8961e-01, 2.7432e-03, 3.6698e-03, 5.5705e-04],
        [5.167

# Load Model for Testing

In [ ]:
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification

OUTPUT_PATH = PATH + "./my_indodistilbert2_topic_classification"

# config = AutoConfig.from_pretrained(OUTPUT_PATH + "config.json")
model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_PATH)
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import AutoTokenizer, DistilBertTokenizer

# tokenizer = AutoTokenizer.from_pretrained(OUTPUT_PATH)
# tokenizer
# model_name = "cahya/distilbert-base-indonesian"
model_name = "cahya/distilbert-base-indonesian"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [ ]:
import pandas as pd
import torch

tokenized_input = tokenizer(test_dataset['text'], return_tensors="pt", truncation=True, padding=True)
tokenized_input

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    3,  1980,  2651,  ...,     2,     2,     2],
        [    3,  4852,  1015,  ...,     2,     2,     2],
        [    3,  8043, 29391,  ...,     2,     2,     2],
        ...,
        [    3, 18586,  3921,  ...,     2,     2,     2],
        [    3, 10648,  2992,  ...,     2,     2,     2],
        [    3,  6959,  1016,  ...,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
model.eval()

# Forward pass
with torch.no_grad():
    outputs = model(**tokenized_input)

    # The logits (raw model output)
    logits = outputs.logits

    predicted_class = torch.argmax(logits, dim=-1)

# Decode label (optional)
id2label = model.config.id2label

for text, pred in zip(test_dataset['labels'], predicted_class):
    label = id2label.get(pred.item(), pred.item())
    print(f"Text: {text}\nPredicted Label: {label}\n")

Text: hidup luas melulu urus politik coba untk ber empati mahluk laen
Predicted Label: LABEL_0

Text: weh wkwk
Predicted Label: LABEL_1

Text: over cringe
Predicted Label: LABEL_0

Text: lambung kaya sempet gitu makan muntah nyoba minum
Predicted Label: LABEL_0

Text: bayang gempa skala richter gempa amat lewat
Predicted Label: LABEL_1

Text: selamat gempa tsunami palu hidup mukjizat orang oran
Predicted Label: LABEL_1

Text: help freebies karna ticket green maw frib pas tanggal ice nanti plac
Predicted Label: LABEL_1

Text: halo ajak teman bantu korban gempa tsunami palu donggala donasi
Predicted Label: LABEL_1

Text: pandai jaga muka jaga kaki share rutin gosok kaki scrubt ive
Predicted Label: LABEL_2

Text: haaaai resmi masuk ptn udah janji giveaway hadiah sunscreen skin spf mas
Predicted Label: LABEL_1

Text: nama lia lumba
Predicted Label: LABEL_1

Text: perempuan perempuan mesti tau sakit orang sayang layan pempuan melaink
Predicted Label: LABEL_0

Text: dunia milyar juta orang t

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
labels = test_dataset['labels']

print(f"acc : {accuracy_score(labels, predicted_class)}")
print(f"f1  : {f1_score(labels, predicted_class, average='weighted')}")

acc : 0.525
f1  : 0.5295269632552236
